### This notebook loads and train data from the directories without having them to load on memory

In [11]:
import os
os.chdir('/home/ak/Spring2018/ature')
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras.callbacks import ModelCheckpoint

import neuralnet.utils.data_utils as dutils
from neuralnet.utils.keras_dataset import KerasPatchesGenerator
from commons.IMAGE import Image
from PIL import Image as IMG
import utils.img_utils as imgutils
import neuralnet.utils.keras_vizual_callbacks as kvz

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Directories setup

In [12]:
### Define folders. Create if needed.
sep = os.sep
Dirs = {}
Dirs['checkpoint']   = 'assests' +sep+ 'nnet_models'
Dirs['data']      = 'data'+sep+'DRIVE'+sep+'training'
Dirs['images']    = Dirs['data'] +sep+ 'images1'
Dirs['mask']      = Dirs['data'] +sep+ 'mask'
Dirs['truth']     = Dirs['data'] +sep+ '1st_manual'
Dirs['segmented'] = Dirs['data'] +sep+ 'drive_segmented'

TestDirs = {}
TestDirs['data']      = 'data'+sep+'DRIVE'+sep+'test'
TestDirs['images']    = TestDirs['data'] +sep+ 'images1'
TestDirs['mask']      = TestDirs['data'] +sep+ 'mask'
TestDirs['truth']     = TestDirs['data'] +sep+ '1st_manual'
TestDirs['segmented'] = TestDirs['data'] +sep+ 'drive_segmented'

ValidationDirs = {}
ValidationDirs['data']      = 'data'+sep+'DRIVE'+sep+'test'
ValidationDirs['images']    = ValidationDirs['data'] +sep+ 'images1'
ValidationDirs['mask']      = ValidationDirs['data'] +sep+ 'mask'
ValidationDirs['truth']     = ValidationDirs['data'] +sep+ '1st_manual'
ValidationDirs['segmented'] = ValidationDirs['data'] +sep+ 'drive_segmented'

for k, folder in Dirs.items():
    os.makedirs(folder, exist_ok=True)

def get_mask_file(file_name): 
    return file_name.split('_')[0] + '_training_mask.gif'

def get_ground_truth_file(file_name): 
    return file_name.split('_')[0] + '_manual1.gif'

def get_segmented_file(file_name):
    return file_name + '_SEG.PNG'

def get_mask_file_test(file_name): 
    return file_name.split('_')[0] + '_test_mask.gif'

classes = {
    'background': 0,
    'vessel': 1,
}
batch_size = 52
num_classes = len(classes)
epochs = 2
patch_size = 31

### Define data sources

In [13]:
traingen = ImageDataGenerator(
    rotation_range=70,
    horizontal_flip=True,
    samplewise_center=True,
    samplewise_std_normalization=True,
    vertical_flip=True
)

testgen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
)

train_generator = KerasPatchesGenerator(Dirs=Dirs, patch_size=patch_size, 
                                   num_classes=num_classes,
                                   fget_mask=get_mask_file, 
                                   fget_truth=get_ground_truth_file,
                                   fget_segmented=get_segmented_file,
                                   batch_size=batch_size, img_generator=traingen, shuffle=True)

validation_generator = KerasPatchesGenerator(Dirs=ValidationDirs, patch_size=patch_size, 
                                   num_classes=num_classes,
                                   fget_mask=get_mask_file_test, 
                                   fget_truth=get_ground_truth_file,
                                   fget_segmented=get_segmented_file,
                                   batch_size=batch_size, img_generator=testgen)

test_generator = KerasPatchesGenerator(Dirs=TestDirs, patch_size=patch_size, 
                                   num_classes=num_classes,
                                   fget_mask=get_mask_file_test, 
                                   fget_truth=get_ground_truth_file,
                                   fget_segmented=get_segmented_file,
                                   batch_size=batch_size, img_generator=testgen)

### 4192 batches found.
### 4168 batches found.
### 4168 batches found.


### Define model

In [18]:
model = Sequential() 

model.add(Conv2D(52, (3, 3), padding='same', input_shape=(patch_size, patch_size, 1)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [19]:
# initiate Adam optimizer
opt = keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Let's train the model using Adam
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=Dirs['checkpoint'] + 'keras_model_checkpoint.hdf5', verbose=1, save_best_only=True)
callbacks = [checkpointer]

### Fit model from directory generators

In [20]:
class_weights = dutils.get_class_weights(np.array(train_generator.IDs)[:,3])
model.fit_generator(
        train_generator,
        steps_per_epoch=1000,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=800, verbose=1,
        callbacks=callbacks, workers=5)

Epoch 1/2
1000/1000 [==============================] - 423s 423ms/step - loss: 0.3617 - acc: 0.8910 - val_loss: 0.4645 - val_acc: 0.8312

Epoch 00001: val_loss improved from inf to 0.46449, saving model to assests/nnet_modelskeras_model_checkpoint.hdf5
Epoch 2/2
  16/1000 [..............................] - ETA: 4:50 - loss: 0.3290 - acc: 0.8930

KeyboardInterrupt: 

In [135]:
### If we shuffle, we won't be able to generate a segmented image.
prediction = model.predict_generator(test_generator, workers=3)
pred = np.argmax(prediction, axis=1)

KeyboardInterrupt: 

In [8]:
img_obj = Image()
img_obj.load_file(data_dir=Dirs['images'], file_name='21_training.tif')
img_obj.working_arr = img_obj.image_arr[:, :, 1]

In [36]:
seg = np.zeros_like(img_obj.working_arr)

In [ ]:
for ix, IDij in enumerate(test_generator.IDs, 0):
    ID, i, j = IDij
    try:
        if pred[ix]==classes['white'] or pred[ix]==classes['red']:
            seg[int(i),int(j)] = 255
    except:
        print('Ouch!', end='\r')
IMG.fromarray(seg)

In [37]:
# SANITY CHECK
for ID, i, j, y in train_generator.IDs:
    if ID==0:
        seg[i,j] = 255 if y == 0 or y == 3 else seg[i,j]

In [22]:
test_generator.shuffle = True
test_generator.batch_size =1
for x, y in test_generator:
    m = model.predict_classes(x)
    if m[0]==1:
        print(m)
#     pass

KeyboardInterrupt: 

In [30]:
jj = np.squeeze(x)

In [31]:
jj.shape

(31, 31)

In [32]:
jj = imgutils.rescale2d_unsigned(jj) * 255

In [33]:
IMG.fromarray(np.array(jj, dtype=np.uint8))

In [7]:
print(num_classes)

2


In [23]:
x = [1,2,3,4]

In [24]:
x.pop()

4

AttributeError: 'list' object has no attribute 'push'

In [30]:
print('%s'%('haha'))

haha


In [32]:
x = np.full((20,20), 50)

In [35]:
y = [0,3,4,5,6]
z = [10,20,31,55]

In [36]:
x[y] = z

ValueError: shape mismatch: value array of shape (4,) could not be broadcast to indexing result of shape (5,20)